In [1]:
import pandas as pd
import pymer4.models
import numpy as np
import scipy.stats

In [2]:
measures = pd.read_csv("measures.csv")
measures["RT_z"] = (measures["RT"]-measures["RT"].mean())/measures["RT"].std()
measures["d_condition_z"] = (measures["d_condition"]-measures["d_condition"].mean())/measures["d_condition"].std()
measures["tta_condition_z"] = (measures["tta_condition"]-measures["tta_condition"].mean())/measures["tta_condition"].std()
measures["tta_or_condition_z"] = (measures["tta_or_condition"]-measures["tta_or_condition"].mean())/measures["tta_or_condition"].std()
measures["dwell_time_z"] = (measures["dwell_time"]-measures["dwell_time"].mean())/measures["dwell_time"].std()

In [3]:
column_names = {"Estimate": "$\\beta$", "Z-stat": "$z$", "P-val": "$p$", "T-stat": "$t$", "F-stat": "$F$", "2.5_ci": "CI 2.5\%", "97.5_ci": "CI 97.5\%", "NumDF": "df"}
var_names = {"tta_condition_z": "$\\textrm{TTA}$",
             "d_condition_z": "distance",
             "tta_or_condition_z": "time budget",
             "dwell_time_z": "\% dwell time mirror",
             "tta_condition_z:tta_or_condition_z": "$\\textrm{TTA}$:time budget",
             "tta_condition_z:dwell_time_z": "$\\textrm{TTA}$:\% dwell time mirror",
             "tta_or_condition_z:dwell_time_z": "time budget:\% dwell time mirror",
             "tta_condition_z:tta_or_condition_z:dwell_time_z": "$\\textrm{TTA}$:time budget:\% dwell time mirror",
             "decision:d_condition_z": "decision:distance",
             "decision:tta_condition_z": "decision:$\\textrm{TTA}$",
             "decision:tta_or_condition_z": "decision:time budget",
             "decision:tta_condition_z:tta_or_condition_z": "decision:$\\textrm{TTA}$:time budget"}

def p_formatted(p):
    if p>0.01:
        return "{:.2f}".format(p)
    elif p>0.001:
        return "{:.3f}".format(p)
    else:
        return "$<0.001$"

# Dwell time

In [12]:
model_dwell_time = pymer4.models.Lmer("dwell_time_z ~ RT_z*decision + (1 | subj_id) ", data=measures, family="gaussian")
model_dwell_time_fit = model_dwell_time.fit(summarize=True)
model_dwell_time.coefs

Linear mixed model fit by REML [’lmerMod’]
Formula: dwell_time_z~RT_z*decision+(1|subj_id)

Family: gaussian	 Inference: parametric

Number of observations: 8634	 Groups: {'subj_id': 24.0}

Log-likelihood: -8890.112 	 AIC: 17792.225

Random effects:

                 Name    Var    Std
subj_id   (Intercept)  0.486  0.697
Residual               0.451  0.671

No random effect correlations specified

Fixed effects:


,Estimate,2.5_ci,97.5_ci,SE,DF,T-stat,P-val,Sig
(Intercept),-0.038547,-0.318179,0.241084,0.142672,23.079816,-0.270182,7.894213e-01,
RT_z,-0.195625,-0.220385,-0.170864,0.012633,8625.113433,-15.485115,2.276965e-53,***
decisionWait,0.082622,0.046667,0.118577,0.018345,8613.065527,4.503848,6.761239e-06,***
RT_z:decisionWait,0.054404,0.020695,0.088114,0.017199,8612.813949,3.163214,1.565800e-03,**


# Decision outcome

In [53]:
model_decision = pymer4.models.Lmer("is_gap_accepted ~ 1 + d_condition_z + tta_condition_z + tta_or_condition_z + (1 + tta_condition_z + tta_or_condition_z + d_condition_z | subj_id)", data=measures, family="binomial")
model_decision_fit = model_decision.fit(summarize=True)
model_decision.coefs

Linear mixed model fit by maximum likelihood  ['lmerMod']
Formula: is_gap_accepted~1+d_condition_z+tta_condition_z+tta_or_condition_z+(1+tta_condition_z+tta_or_condition_z+d_condition_z|subj_id)

Family: binomial	 Inference: parametric

Number of observations: 8634	 Groups: {'subj_id': 24.0}

Log-likelihood: -3232.833 	 AIC: 6493.667

Random effects:

                       Name    Var    Std
subj_id         (Intercept)  1.621  1.273
subj_id     tta_condition_z  0.268  0.518
subj_id  tta_or_condition_z  0.202  0.449
subj_id       d_condition_z  0.251  0.501

                        IV1                 IV2   Corr
subj_id         (Intercept)     tta_condition_z -0.124
subj_id         (Intercept)  tta_or_condition_z -0.220
subj_id         (Intercept)       d_condition_z  0.215
subj_id     tta_condition_z  tta_or_condition_z -0.140
subj_id     tta_condition_z       d_condition_z -0.548
subj_id  tta_or_condition_z       d_condition_z  0.462

Fixed effects:


,Estimate,2.5_ci,97.5_ci,SE,OR,OR_2.5_ci,OR_97.5_ci,Prob,Prob_2.5_ci,Prob_97.5_ci,Z-stat,P-val,Sig
(Intercept),1.299452,0.780160,1.818745,0.264950,3.667288,2.181821,6.164116,0.785743,0.685715,0.860415,4.904520,9.365625e-07,***
d_condition_z,0.537341,0.325559,0.749123,0.108054,1.711451,1.384805,2.115145,0.631194,0.580678,0.678988,4.972894,6.596063e-07,***
tta_condition_z,1.841681,1.611953,2.071408,0.117210,6.307130,5.012592,7.935991,0.863147,0.833682,0.888093,15.712650,1.238910e-55,***
tta_or_condition_z,-0.416692,-0.608406,-0.224978,0.097815,0.659224,0.544218,0.798534,0.397309,0.352423,0.443992,-4.259998,2.044290e-05,***


In [56]:
df = model_decision.coefs.loc[:, ["Estimate", "SE", "Z-stat", "P-val"]]

df["P-val"] = df["P-val"].apply(p_formatted)

styler = df.rename(columns=column_names, index=var_names).style.format(precision=2)

with open('C:\\Users\\azgonnikov\\Dropbox\\Apps\\Overleaf\\AAP Dynamics of merging decisions Elsevier template\\tab_decision.tex', 'w') as tf:
     tf.write(styler.to_latex(
         column_format="rrrrr", position="h", position_float="centering",
         hrules=True, label="tab:decision", caption="Standardized coefficients of the mixed-effects logistic regression describing the final decision as a function of $\\textrm{TTA}$ and distance to the overtaking vehicle and the time budget provided by the merging lane. All effects were modelled as random slopes per participant: \\texttt{decision $\sim$ 1 + distance + TTA + time budget + (1 + distance + TTA + time budget) | participant}."
     )
)

# Response time

In [64]:
model_RT = pymer4.models.Lmer("RT_z ~ 1 + decision*(d_condition_z + tta_condition_z*tta_or_condition_z) + (decision | subj_id) ", data=measures, family="gaussian")
model_RT.fit(summarize=True, factors={"decision": ["Merge", "Wait"]})
model_RT.coefs
RT_anova = model_RT.anova()
RT_decision_marginal_estimates, RT_decision_comparisons = model_RT.post_hoc(marginal_vars=["decision"])

Linear mixed model fit by REML [’lmerMod’]
Formula: RT_z~1+decision*(d_condition_z+tta_condition_z*tta_or_condition_z)+(decision|subj_id)

Family: gaussian	 Inference: parametric

Number of observations: 8634	 Groups: {'subj_id': 24.0}

Log-likelihood: -9164.353 	 AIC: 18356.706

Random effects:

                  Name    Var    Std
subj_id    (Intercept)  0.343  0.586
subj_id   decisionWait  0.201  0.449
Residual                0.475  0.689

                 IV1           IV2   Corr
subj_id  (Intercept)  decisionWait -0.431

Fixed effects:

SS Type III Analysis of Variance Table with Satterthwaite approximated degrees of freedom:
(NOTE: Using original model contrasts, orthogonality not guaranteed)


In [59]:
RT_anova

,SS,MS,NumDF,DenomDF,F-stat,P-val,Sig
decision,61.469085,61.469085,1,23.988530,129.529043,3.732783e-11,***
d_condition_z,0.599919,0.599919,1,8594.728326,1.264164,2.608958e-01,
tta_condition_z,129.168121,129.168121,1,8597.057316,272.185979,3.173882e-60,***
tta_or_condition_z,63.368563,63.368563,1,8586.658891,133.531667,1.165224e-30,***
tta_condition_z:tta_or_condition_z,18.071613,18.071613,1,8582.008688,38.080911,7.094581e-10,***
decision:d_condition_z,15.262386,15.262386,1,8591.988757,32.161243,1.464876e-08,***
decision:tta_condition_z,79.633941,79.633941,1,8599.404593,167.806438,5.051654e-38,***
decision:tta_or_condition_z,18.125095,18.125095,1,8583.265824,38.193609,6.698096e-10,***
decision:tta_condition_z:tta_or_condition_z,1.861408,1.861408,1,8581.322751,3.922400,4.767869e-02,*


In [60]:
df = RT_anova.loc[:, ["SS", "MS", "F-stat", "P-val"]]
df["P-val"] = df["P-val"].apply(p_formatted)
styler = df.rename(columns=column_names, index=var_names).style.format(precision=2)

with open('C:\\Users\\azgonnikov\\Dropbox\\Apps\\Overleaf\\AAP Dynamics of merging decisions Elsevier template\\tab_RT.tex', 'w') as tf:
     tf.write(styler.to_latex(
         column_format="rrrrrr", position="h", position_float="centering",
         hrules=True, label="tab:RT", caption="ANOVA table based on the mixed-effects linear regression describing response time as a function of TTA and distance to the overtaking vehicle and the time budget provided by the merging lane. Random slope of decision was included per participant: \\texttt{RT $\sim$ 1 + decision*(TTA*time budget + distance) + (1 + decision) | participant}."
     )
)

## Difference between accept and reject RTs

In [65]:
# RT_decision_marginal_estimates, RT_decision_comparisons = model_RT.post_hoc(marginal_vars=["tta_condition_z"], grouping_vars=["decision"])
RT_decision_marginal_estimates, RT_decision_comparisons = model_RT.post_hoc(marginal_vars=["decision"])
RT_decision_comparisons

,Contrast,Estimate,2.5_ci,97.5_ci,SE,DF,T-stat,P-val,Sig
1,Merge - Wait,-1.077,-1.272,-0.881,0.095,23.988,-11.382,0.0,***


In [66]:
RT_decision_comparisons.Estimate*measures.RT.std()

1   -0.687359
Name: Estimate, dtype: float64

## Estimates of condition effects on RT per decision

In [67]:
def get_marginal_estimates(model_RT, marginal_vars):
    marginal_estimates, comparisons = model_RT.post_hoc(marginal_vars=marginal_vars, grouping_vars=["decision"])
    marginal_estimates["T-stat"] = marginal_estimates["Estimate"]/marginal_estimates["SE"]
    marginal_estimates["P-val"] = scipy.stats.t.sf(np.abs(marginal_estimates["T-stat"]), marginal_estimates.DF)
    return marginal_estimates

In [68]:
get_marginal_estimates(model_RT, "tta_condition_z")

,decision,Estimate,2.5_ci,97.5_ci,SE,DF,T-stat,P-val
1,Merge,0.039,0.019,0.059,0.010,8589.916,3.900000,4.846282e-05
2,Wait,0.324,0.286,0.362,0.019,8598.869,17.052632,1.880770e-64


In [69]:
get_marginal_estimates(model_RT, "d_condition_z")

,decision,Estimate,2.5_ci,97.5_ci,SE,DF,T-stat,P-val
1,Merge,-0.037,-0.055,-0.018,0.009,8579.888,-4.111111,0.000020
2,Wait,0.055,0.029,0.081,0.013,8597.518,4.230769,0.000012


In [70]:
get_marginal_estimates(model_RT, "tta_or_condition_z")

,decision,Estimate,2.5_ci,97.5_ci,SE,DF,T-stat,P-val
1,Merge,0.057,0.038,0.077,0.010,8579.983,5.700000,6.188545e-09
2,Wait,0.189,0.152,0.226,0.019,8586.301,9.947368,1.727870e-23


## Estimates of time budget effect per decision and TTA level

In [71]:
marginal_estimates, comparisons = model_RT.post_hoc(marginal_vars=["tta_or_condition_z"], grouping_vars=["decision", "tta_condition_z"])
marginal_estimates["T-stat"] = marginal_estimates["Estimate"]/marginal_estimates["SE"]
marginal_estimates["P-val"] = scipy.stats.t.sf(np.abs(marginal_estimates["T-stat"]), marginal_estimates.DF)

In [72]:
marginal_estimates

,decision,tta_condition_z,Estimate,2.5_ci,97.5_ci,SE,DF,T-stat,P-val
1,Merge,-1.0,0.013,-0.020,0.046,0.017,8580.044,0.764706,2.222339e-01
2,Wait,-1.0,0.103,0.075,0.130,0.014,8590.524,7.357143,1.025880e-13
3,Merge,1.0,0.102,0.080,0.123,0.011,8578.325,9.272727,1.129338e-20
4,Wait,1.0,0.276,0.207,0.344,0.035,8583.289,7.885714,1.755460e-15


# Including dwell time in the model for decision outcome

In [4]:
model_decision_dwell = pymer4.models.Lmer("is_gap_accepted ~ d_condition_z + tta_condition_z*tta_or_condition_z*dwell_time_z + (tta_condition_z + d_condition_z + tta_or_condition_z | subj_id) ", data=measures, family="binomial")
model_decision_dwell_fit = model_decision_dwell.fit(summarize=True)
model_decision_dwell.coefs

Linear mixed model fit by maximum likelihood  ['lmerMod']
Formula: is_gap_accepted~d_condition_z+tta_condition_z*tta_or_condition_z*dwell_time_z+(tta_condition_z+d_condition_z+tta_or_condition_z|subj_id)

Family: binomial	 Inference: parametric

Number of observations: 8634	 Groups: {'subj_id': 24.0}

Log-likelihood: -3213.004 	 AIC: 6464.008

Random effects:

                       Name    Var    Std
subj_id         (Intercept)  1.657  1.287
subj_id     tta_condition_z  0.261  0.511
subj_id       d_condition_z  0.253  0.503
subj_id  tta_or_condition_z  0.114  0.338

                     IV1                 IV2   Corr
subj_id      (Intercept)     tta_condition_z -0.132
subj_id      (Intercept)       d_condition_z  0.219
subj_id      (Intercept)  tta_or_condition_z -0.138
subj_id  tta_condition_z       d_condition_z -0.540
subj_id  tta_condition_z  tta_or_condition_z -0.206
subj_id    d_condition_z  tta_or_condition_z  0.580

Fixed effects:


,Estimate,2.5_ci,97.5_ci,SE,OR,OR_2.5_ci,OR_97.5_ci,Prob,Prob_2.5_ci,Prob_97.5_ci,Z-stat,P-val,Sig
(Intercept),1.277144,0.751927,1.802361,0.267973,3.586382,2.121083,6.063945,0.781963,0.679598,0.858436,4.765948,1.879680e-06,***
d_condition_z,0.542184,0.329556,0.754812,0.108486,1.719759,1.390351,2.127212,0.632320,0.581651,0.680226,4.997754,5.800192e-07,***
tta_condition_z,1.828735,1.601117,2.056353,0.116134,6.226004,4.958567,7.817406,0.861611,0.832174,0.886588,15.746795,7.225156e-56,***
tta_or_condition_z,-0.391190,-0.546368,-0.236013,0.079174,0.676251,0.579049,0.789770,0.403431,0.366707,0.441269,-4.940920,7.775490e-07,***
dwell_time_z,0.062410,-0.037146,0.161967,0.050795,1.064399,0.963535,1.175821,0.515597,0.490714,0.540403,1.228663,2.191980e-01,
tta_condition_z:tta_or_condition_z,0.049566,-0.032159,0.131291,0.041697,1.050815,0.968353,1.140299,0.512389,0.491961,0.532776,1.188719,2.345503e-01,
tta_condition_z:dwell_time_z,0.066278,-0.029596,0.162151,0.048916,1.068523,0.970838,1.176038,0.516563,0.492602,0.540449,1.354929,1.754400e-01,
tta_or_condition_z:dwell_time_z,0.239457,0.150385,0.328528,0.045445,1.270559,1.162282,1.388922,0.559580,0.537526,0.581401,5.269113,1.370847e-07,***
tta_condition_z:tta_or_condition_z:dwell_time_z,-0.046320,-0.121023,0.028382,0.038114,0.954736,0.886014,1.028789,0.488422,0.469781,0.507095,-1.215300,2.242518e-01,


In [ ]:
df = model_decision_dwell.coefs.loc[:, ["Estimate", "SE", "Z-stat", "P-val"]]

df["P-val"] = df["P-val"].apply(p_formatted)

styler = df.rename(columns=column_names, index=var_names).style.format(precision=2)

with open('C:\\Users\\azgonnikov\\Dropbox\\Apps\\Overleaf\\AAP Dynamics of merging decisions Elsevier template\\tab_decision_dwell.tex', 'w') as tf:
     tf.write(styler.to_latex(
         column_format="rrrrr", position="h", position_float="centering",
         hrules=True, label="tab:decision_dwell", caption="Coefficients of the mixed-effects logistic regression describing the decision outcome as a function of kinematic variables and relative dwell time. Random slopes of TTA and distance to the overtaking vehicle and the time budget provided by the merging lane were included per participant: \\texttt{decision $\sim$ 1 + distance + (TTA * time budget * \\% dwell time mirror) + (1 + distance + TTA + time budget) | participant}."
     )
)

# Including dwell time and RT in the model for decision outcome

In [14]:
model_decision_dwell = pymer4.models.Lmer("is_gap_accepted ~ RT_z + d_condition_z + tta_condition_z*tta_or_condition_z*dwell_time_z + (tta_condition_z + tta_or_condition_z | subj_id) ", data=measures, family="binomial")
model_decision_dwell_fit = model_decision_dwell.fit(summarize=True)
model_decision_dwell.coefs

Linear mixed model fit by maximum likelihood  ['lmerMod']
Formula: is_gap_accepted~RT_z+d_condition_z+tta_condition_z*tta_or_condition_z*dwell_time_z+(tta_condition_z+tta_or_condition_z|subj_id)

Family: binomial	 Inference: parametric

Number of observations: 8634	 Groups: {'subj_id': 24.0}

Log-likelihood: -2506.973 	 AIC: 5045.945

Random effects:

                       Name    Var    Std
subj_id         (Intercept)  2.418  1.555
subj_id     tta_condition_z  0.597  0.772
subj_id  tta_or_condition_z  0.089  0.298

                     IV1                 IV2   Corr
subj_id      (Intercept)     tta_condition_z  0.443
subj_id      (Intercept)  tta_or_condition_z -0.339
subj_id  tta_condition_z  tta_or_condition_z -0.202

Fixed effects:


,Estimate,2.5_ci,97.5_ci,SE,OR,OR_2.5_ci,OR_97.5_ci,Prob,Prob_2.5_ci,Prob_97.5_ci,Z-stat,P-val,Sig
(Intercept),1.663192,1.027145,2.299238,0.324519,5.276125,2.793081,9.966587,0.840666,0.736362,0.908814,5.125092,2.973929e-07,***
RT_z,-1.962961,-2.085965,-1.839958,0.062758,0.140442,0.124187,0.158824,0.123147,0.110468,0.137056,-31.278204,9.235252e-215,***
d_condition_z,0.506926,0.433030,0.580822,0.037703,1.660180,1.541923,1.787506,0.624086,0.606597,0.641256,13.445386,3.276920e-41,***
tta_condition_z,2.173222,1.836484,2.509960,0.171808,8.786550,6.274438,12.304440,0.897819,0.862532,0.924837,12.649109,1.131509e-36,***
tta_or_condition_z,-0.189079,-0.339501,-0.038656,0.076748,0.827721,0.712125,0.962081,0.452871,0.415931,0.490337,-2.463643,1.375330e-02,*
dwell_time_z,-0.335440,-0.459929,-0.210952,0.063516,0.715023,0.631329,0.809813,0.416918,0.387003,0.447457,-5.281213,1.283313e-07,***
tta_condition_z:tta_or_condition_z,0.159515,0.065350,0.253680,0.048044,1.172942,1.067533,1.288760,0.539794,0.516332,0.563082,3.320177,8.996034e-04,***
tta_condition_z:dwell_time_z,0.101408,-0.018611,0.221428,0.061236,1.106729,0.981561,1.247857,0.525330,0.495347,0.555132,1.656039,9.771382e-02,.
tta_or_condition_z:dwell_time_z,0.233493,0.128505,0.338481,0.053566,1.263004,1.137127,1.402815,0.558109,0.532082,0.583822,4.358954,1.306857e-05,***
tta_condition_z:tta_or_condition_z:dwell_time_z,-0.011502,-0.099915,0.076910,0.045109,0.988563,0.904914,1.079945,0.497124,0.475042,0.519218,-0.254991,7.987297e-01,


In [17]:
df = model_decision_dwell.coefs.loc[:, ["Estimate", "SE", "Z-stat", "P-val"]]

df["P-val"] = df["P-val"].apply(p_formatted)

styler = df.rename(columns=column_names, index=var_names).style.format(precision=2)

with open('C:\\Users\\azgonnikov\\Dropbox\\Apps\\Overleaf\\AAP Dynamics of merging decisions Elsevier template\\tab_decision_dwell_RT.tex', 'w') as tf:
     tf.write(styler.to_latex(
         column_format="rrrrr", position="h", position_float="centering",
         hrules=True, label="tab:decision_dwell", caption="Coefficients of the mixed-effects logistic regression describing the decision outcome as a function of kinematic variables, response time, and relative dwell time. Random slopes of TTA to the overtaking vehicle and the time budget provided by the merging lane were included per participant: \\texttt{decision $\sim$ 1 + distance + (TTA * time budget * \\% dwell time mirror) + (1 + TTA + time budget) | participant}."
     )
)